In [1]:
import src.models.builders as mb
import src.models.helpers as mh
import src.inference.helpers as ih
import plotly.graph_objects as go
import src.modelling_o2.fit_ho2sat_ar as fit_ho2sat_ar
import src.models.point_in_time as point_in_time

import numpy as np

In [2]:
height = 175
age = 30
sex = "Male"
hfev1_prior = {"type": "default", "height": height, "age": age, "sex": sex}
ho2sat_prior = {
    "type": "default",
    "height": height,
    "sex": sex,
}
(
    model,
    inf_alg,
    HFEV1,
    ecFEV1,
    AR,
    HO2Sat,
    O2SatFFA,
    IA,
    UO2Sat,
    O2Sat,
) = point_in_time.build(hfev1_prior, ho2sat_prior)
# ) = model.calc_cpts(hfev1_prior, ho2sat_prior)

## HFEV1 prior

In [33]:
fig = go.Figure()
fig.add_trace(go.Bar(y=HFEV1.prior[:, 0], x=HFEV1.bins))
# fig.add_trace(go.Histogram(x=np.repeat(bins, p), histnorm='probability', nbinsx=len(HFEV1.bins)))
# fig.add_trace(go.Histogram(x=np.repeat(HFEV1.bins, (HFEV1.prior[:,0]*10000).astype(int)), histnorm='probability', xbins=dict(
#         start=0,
#         end=6,
#         size=HFEV1.bin_width
#     )))
fig.update_layout(bargap=0.01)
fig.update_xaxes(range=[0, 6])
fig.update_traces(marker_line_width=0)
fig.show()

## IA drop's distribution for different AR

In [32]:
# Plot the IA.prior[:, 0] against IA.midbins
def plot_cpt_col(AR_bin_idx):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=IA.midbins, y=IA.prior[:, AR_bin_idx], mode="lines"))
    # Change x axis range to IA.a, IA.b
    fig.update_xaxes(range=[IA.a, IA.b])
    fig.update_layout(
        title=f"P(IA | AR={AR.bins_str[AR_bin_idx]}%)",
        xaxis_title="Inactive alveoli (%)",
        yaxis_title="Probability",
        height=200,
        width=600,
        font=dict(size=10),
    )
    # Reduce margin size
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=30))
    fig.show()


plot_cpt_col(0)
plot_cpt_col(30)
plot_cpt_col(44)

# Speedup inference

In [152]:
FEV1_obs = 3
O2Sat_obs = 95

res_hfev1 = ih.infer(
    inf_alg, [HFEV1, AR], [[ecFEV1, FEV1_obs], [O2Sat, O2Sat_obs]], joint=False
)
res_ar = ih.infer(inf_alg, [AR], [[ecFEV1, FEV1_obs], [O2Sat, O2Sat_obs]], joint=False)
res_ho2sat = ih.infer(
    inf_alg, [HO2Sat, AR], [[ecFEV1, FEV1_obs], [O2Sat, O2Sat_obs]], joint=False
)
res_o2satffa = ih.infer(
    inf_alg, [O2SatFFA, AR], [[ecFEV1, FEV1_obs], [O2Sat, O2Sat_obs]], joint=False
)
res_ia = ih.infer(
    inf_alg, [IA, AR], [[ecFEV1, FEV1_obs], [O2Sat, O2Sat_obs]], joint=False
)
res_uo2sat = ih.infer(
    inf_alg, [UO2Sat, AR], [[ecFEV1, FEV1_obs], [O2Sat, O2Sat_obs]], joint=False
)

/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:801: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi1.values


In [24]:
# q = ih.infer(
#     inf_alg,
#     [HFEV1, AR, HO2Sat, IA],
#     [[ecFEV1, 3], [O2Sat, 85]],
#     show_progress=False,
#     joint=False,
# )
# q1 = ih.infer(
#     inf_alg,
#     [AR, O2SatFFA, UO2Sat],
#     [[ecFEV1, 3], [O2Sat, 85]],
#     show_progress=False,
#     joint=False,
# )
# q = ih.infer(inf_alg, [O2SatFFA, HFEV1], [[ecFEV1, 3], [O2Sat, 85]], show_progress=False, joint=False) # 4k, 0.1s
q = ih.infer(inf_alg, [O2SatFFA, HFEV1, AR], [[ecFEV1, 3], [O2Sat, 85]], show_progress=False, joint=False) # 180, 0.1s
# q = ih.infer(inf_alg, [O2SatFFA, HFEV1, AR, HO2Sat], [[ecFEV1, 3], [O2Sat, 85]], show_progress=False, joint=False) # 3.6m, 0.1s
# q = ih.infer(inf_alg, [O2SatFFA, HFEV1, AR, HO2Sat, IA], [[ecFEV1, 3], [O2Sat, 85]], show_progress=False, joint=False) # 108m, 4.4s
# q = ih.infer(inf_alg, [O2SatFFA, HFEV1, UO2Sat], [[ecFEV1, 3], [O2Sat, 85]], show_progress=False, joint=False) # 400k, 5.7s
# q = ih.infer(inf_alg, [O2SatFFA, HFEV1, UO2Sat, AR], [[ecFEV1, 3], [O2Sat, 85]], show_progress=False, joint=False) # 18m, 1s
# q = ih.infer(inf_alg, [O2SatFFA, HFEV1, UO2Sat, AR, HO2Sat, IA], [[ecFEV1, 3], [O2Sat, 85]], show_progress=False, joint=False) # 10.8b, crashes
# q = ih.infer(inf_alg, [O2SatFFA, HFEV1, UO2Sat, AR, IA], [[ecFEV1, 3], [O2Sat, 85]], show_progress=False, joint=False) # 540m, 2min 26s
q

/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:801: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi1.values


{'O2 saturation if fully functional alveoli (%)': <DiscreteFactor representing phi(O2 saturation if fully functional alveoli (%):40) at 0x7ff5c395e4a0>,
 'Healthy FEV1 (L)': <DiscreteFactor representing phi(Healthy FEV1 (L):100) at 0x7ff5c395eda0>,
 'Airway resistance (%)': <DiscreteFactor representing phi(Airway resistance (%):45) at 0x7ff5c395c940>}

Why does it takes more time to infer HFEV1 alone rather than HFEV1, AR, HO2Sat, O2SatFFA at once?


In [86]:
q["Airway resistance (%)"] == q1["Airway resistance (%)"]

True

# Speedup model building by saving all CPTs
Priors for HFEV1 and HO2Sat depend on age, sex, height. 
CPTs can be locally saved. As long as we don't change the variables binning they remain the same

In [1]:
# Save ecFEV1 prior to ../cpts/ecfev1_prior.txt
np.savetxt(
    f"../models/cpts/ecFEV1_{ecFEV1.a}_{ecFEV1.b}_{ecFEV1.bin_width}_HFEV1_{HFEV1.a}_{HFEV1.b}_{HFEV1.bin_width}_AR_{AR.a}_{AR.b}_{AR.bin_width}.txt",
    ecFEV1.prior,
    delimiter=",",
)
# Save AR no need it's uniform
# Save O2SatFFA
np.savetxt(
    f"../models/cpts/O2SatFFA_{O2SatFFA.a}_{O2SatFFA.b}_{O2SatFFA.bin_width}_HO2SAt_{HO2Sat.a}_{HO2Sat.b}_{HO2Sat.bin_width}_AR_{AR.a}_{AR.b}_{AR.bin_width}.txt",
    O2SatFFA.prior,
    delimiter=",",
)
# Save IA
np.savetxt(
    f"../models/cpts/IA_{IA.a}_{IA.b}_{IA.bin_width}_AR_{AR.a}_{AR.b}_{AR.bin_width}.txt",
    IA.prior,
    delimiter=",",
)
# Save UO2Sat
np.savetxt(
    f"../models/cpts/UO2Sat_{UO2Sat.a}_{UO2Sat.b}_{UO2Sat.bin_width}_O2SatFFA_{O2SatFFA.a}_{O2SatFFA.b}_{O2SatFFA.bin_width}_IA_{IA.a}_{IA.b}_{IA.bin_width}.txt",
    UO2Sat.prior,
    delimiter=",",
)
# Save O2Sat
np.savetxt(
    f"../models/cpts/O2Sat_{O2Sat.a}_{O2Sat.b}_{O2Sat.bin_width}_UO2Sat_{UO2Sat.a}_{UO2Sat.b}_{UO2Sat.bin_width}.txt",
    O2Sat.prior,
    delimiter=",",
)

NameError: name 'np' is not defined

In [3]:
import src.models.cpts.load as cptload

cptload.get_cpt([ecFEV1, HFEV1, AR])

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])